In [ ]:
# ----------------------------------------------------
# 🧹 "최종 데이터 정제 스크립트" (이것만 단독으로 실행하세요)
# ----------------------------------------------------
import joblib
import pandas as pd
import numpy as np
import os

print("="*80)
print("🧹 데이터 정제: Flow-Packet 동기화 중복 제거")
print("="*80)

# 1. 원본 데이터 로딩
print("📂 1단계: 원본 데이터 로딩...")
flow_data = joblib.load("task2_data/train_flow_data.pkl")

all_packets = []
packet_files = [f"task2_data/train_packet_data_{i}.pkl" for i in range(50000, 650000, 50000)]
for file_path in packet_files:
    if os.path.exists(file_path):
        packets_chunk = joblib.load(file_path)
        all_packets.extend(packets_chunk)
print(f"✅ Flow/Packet 데이터 로딩 완료")

# 2. 'Flow' 데이터를 기준으로 중복되지 않은 '원본 인덱스' 확보
print("\n🔍 2단계: 유효 인덱스 식별...")
initial_rows = len(flow_data)
non_duplicate_indices = flow_data.drop_duplicates().index
final_rows = len(non_duplicate_indices)
print(f"✅ 유효 인덱스 {final_rows:,}개 확보 (제거될 중복: {initial_rows - final_rows:,}개)")

# 3. '유효 인덱스'를 사용하여 Flow와 Packet 데이터 동시 정제
print("\n🔄 3단계: 데이터 동기화 정제...")
# 🔥 오류 수정: reset_index(drop=True)를 절대 사용하지 않아, 원본과의 연결을 유지해야 하지만,
# 최종 저장 파일은 0부터 시작하는 인덱스를 갖는 것이 일반적이므로, 여기서는 reset_index를 사용합니다.
# 중요한 것은 Packet 데이터도 동일한 순서로 정렬된 후에 저장된다는 점입니다.
flow_data_cleaned = flow_data.loc[non_duplicate_indices].reset_index(drop=True)
packet_data_cleaned = [all_packets[i] for i in non_duplicate_indices]
print(f"✅ 동기화 완료! 최종 데이터: {len(flow_data_cleaned):,}개")

# 4. 정제된 데이터를 새로운 파일로 저장
print("\n💾 4단계: 정제된 데이터셋 저장...")
cleaned_flow_path = "task2_data/train_flow_data_cleaned.pkl"
cleaned_packet_path = "task2_data/train_packet_data_cleaned.pkl"
joblib.dump(flow_data_cleaned, cleaned_flow_path)
joblib.dump(packet_data_cleaned, cleaned_packet_path)
print(f"✅ 저장 완료: {cleaned_flow_path}, {cleaned_packet_path}")
print(f"\n🎉 모든 데이터 정제 작업 완료!")



In [ ]:
# 🌍 전체 데이터 균등 층화 샘플링 및 모델링
print("="*80)
print("🌍 전체 데이터 균등 층화 샘플링 + 4개 모델 하이퍼파라미터 튜닝")
print("="*80)

import joblib
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from catboost import CatBoostClassifier
import xgboost as xgb
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')
import time
import os

# 1단계: 전체 파일 목록 및 정보 확인
print("\n📂 1단계: 전체 데이터 파일 스캔")
print("="*50)

# 패킷 데이터 파일들 (50000부터 600000까지 50000씩)
packet_files = []
for i in range(50000, 650000, 50000):  # 50000, 100000, ..., 600000
    file_path = f"task2_data/train_packet_data_{i}.pkl"
    if os.path.exists(file_path):
        packet_files.append(file_path)
        print(f"✓ 발견: {file_path}")
    else:
        print(f"❌ 없음: {file_path}")

print(f"\n📊 총 {len(packet_files)}개 패킷 파일 발견")

# Flow 데이터 로딩 (클래스 분포 확인용)
print(f"\n📈 Flow 데이터 로딩 및 클래스 분포 분석...")
flow_data = joblib.load("task2_data/train_flow_data.pkl")
print(f"✓ Flow 데이터 크기: {flow_data.shape}")

# 전체 클래스 분포 확인
duration_dist = flow_data['duration_class'].value_counts().sort_index()
volume_dist = flow_data['volume_class'].value_counts().sort_index()

print(f"\n📊 전체 Duration Class 분포:")
for cls, count in duration_dist.items():
    percentage = (count / len(flow_data)) * 100
    print(f"   Class {int(cls)}: {count:,}개 ({percentage:.1f}%)")

print(f"\n📊 전체 Volume Class 분포:")
for cls, count in volume_dist.items():
    percentage = (count / len(flow_data)) * 100
    print(f"   Class {int(cls)}: {count:,}개 ({percentage:.1f}%)")

# 2단계: 균등 + 층화 샘플링 전략
print(f"\n🎯 2단계: 균등 + 층화 샘플링 전략")
print("="*50)

# 목표 샘플 수 설정
target_total_samples = 50000  # 전체 목표 샘플 수
samples_per_file = target_total_samples // len(packet_files)  # 파일당 균등 샘플 수

print(f"🎯 목표 총 샘플 수: {target_total_samples:,}개")
print(f"📁 파일당 균등 샘플 수: {samples_per_file:,}개")
print(f"📊 층화 추출 기준: Duration + Volume 복합 클래스 비율 보존")

def stratified_sample_from_file(file_path, sample_size, flow_data_subset):
    """
    파일 내에서 층화 추출 수행 - Duration + Volume 복합 층화
    """
    # Duration + Volume 복합 클래스 기준으로 층화 추출
    stratify_key = flow_data_subset['duration_class'].astype(str) + '_' + flow_data_subset['volume_class'].astype(str)
    
    stratified_sampler = StratifiedShuffleSplit(
        n_splits=1, 
        train_size=sample_size, 
        random_state=42
    )
    
    indices = np.arange(len(flow_data_subset))
    stratified_indices, _ = next(stratified_sampler.split(indices, stratify_key))
    
    return stratified_indices

# 3단계: 각 파일에서 균등 + 층화 샘플링
print(f"\n⚖️ 3단계: 각 파일에서 균등 + 층화 샘플링 실행")
print("="*50)

sampled_indices = []
total_sampled = 0

for i, file_path in enumerate(packet_files, 1):
    print(f"\n📁 파일 {i}/{len(packet_files)}: {file_path}")
    
    # 파일 번호 추출 (예: train_packet_data_100000.pkl -> 100000)
    file_num = int(file_path.split('_')[-1].split('.')[0])
    
    # 해당 파일에 대응하는 flow 데이터 범위
    start_idx = (i-1) * 50000
    end_idx = min(start_idx + 50000, len(flow_data))
    flow_subset = flow_data.iloc[start_idx:end_idx].copy()
    flow_subset.reset_index(drop=True, inplace=True)
    
    print(f"   📊 대응 Flow 범위: {start_idx:,} ~ {end_idx-1:,} ({len(flow_subset):,}개)")
    
    # 이 구간의 클래스 분포
    subset_duration_dist = flow_subset['duration_class'].value_counts().sort_index()
    subset_volume_dist = flow_subset['volume_class'].value_counts().sort_index()
    print(f"   🎯 Duration 분포: {dict(subset_duration_dist)}")
    print(f"   📦 Volume 분포: {dict(subset_volume_dist)}")
    
    # 복합 층화 키 분포 확인
    subset_stratify_key = flow_subset['duration_class'].astype(str) + '_' + flow_subset['volume_class'].astype(str)
    subset_combined_dist = subset_stratify_key.value_counts().sort_index()
    print(f"   🔗 복합층(D_V): {len(subset_combined_dist)}개 조합")
    
    # 층화 샘플링 실행
    try:
        actual_sample_size = min(samples_per_file, len(flow_subset))
        stratified_indices = stratified_sample_from_file(file_path, actual_sample_size, flow_subset)
        
        # 전체 인덱스로 변환 (start_idx 더하기)
        global_indices = [start_idx + idx for idx in stratified_indices]
        sampled_indices.extend(global_indices)
        
        total_sampled += len(stratified_indices)
        
        # 샘플링된 데이터의 클래스 분포 확인
        sampled_flow = flow_subset.iloc[stratified_indices]
        sampled_duration_dist = sampled_flow['duration_class'].value_counts().sort_index()
        sampled_volume_dist = sampled_flow['volume_class'].value_counts().sort_index()
        
        print(f"   ✅ 샘플링 완료: {len(stratified_indices):,}개")
        print(f"   📈 샘플 Duration: {dict(sampled_duration_dist)}")
        print(f"   📦 샘플 Volume: {dict(sampled_volume_dist)}")
        
        # 복합 비율 보존 확인
        original_duration_ratios = (subset_duration_dist / len(flow_subset)).round(3)
        sampled_duration_ratios = (sampled_duration_dist / len(stratified_indices)).round(3)
        original_volume_ratios = (subset_volume_dist / len(flow_subset)).round(3)
        sampled_volume_ratios = (sampled_volume_dist / len(stratified_indices)).round(3)
        
        print(f"   ⚖️ Duration 비율보존: 원본{dict(original_duration_ratios)} vs 샘플{dict(sampled_duration_ratios)}")
        print(f"   ⚖️ Volume 비율보존: 원본{dict(original_volume_ratios)} vs 샘플{dict(sampled_volume_ratios)}")
        
    except Exception as e:
        print(f"   ❌ 샘플링 실패: {e}")
        continue

print(f"\n✅ 전체 샘플링 완료!")
print(f"📊 총 샘플링된 인덱스: {len(sampled_indices):,}개")
print(f"🎯 목표 대비 달성률: {len(sampled_indices)/target_total_samples*100:.1f}%")

# 4단계: 샘플링된 데이터의 최종 클래스 분포 확인
print(f"\n📊 4단계: 최종 샘플링 결과 검증")
print("="*50)

# 샘플링된 flow 데이터
sampled_flow_data = flow_data.iloc[sampled_indices].copy()
sampled_flow_data.reset_index(drop=True, inplace=True)

final_duration_dist = sampled_flow_data['duration_class'].value_counts().sort_index()
final_volume_dist = sampled_flow_data['volume_class'].value_counts().sort_index()

print(f"🎯 최종 샘플링된 Duration Class 분포:")
for cls, count in final_duration_dist.items():
    original_ratio = duration_dist[cls] / len(flow_data) * 100
    sampled_ratio = count / len(sampled_flow_data) * 100
    print(f"   Class {int(cls)}: {count:,}개 ({sampled_ratio:.1f}%) [원본: {original_ratio:.1f}%]")

print(f"\n📦 최종 샘플링된 Volume Class 분포:")
for cls, count in final_volume_dist.items():
    original_ratio = volume_dist[cls] / len(flow_data) * 100
    sampled_ratio = count / len(sampled_flow_data) * 100
    print(f"   Class {int(cls)}: {count:,}개 ({sampled_ratio:.1f}%) [원본: {original_ratio:.1f}%]")

# 클래스 비율 보존도 측정 (Duration + Volume 모두)
duration_preservation = []
for cls in final_duration_dist.index:
    original_ratio = duration_dist[cls] / len(flow_data)
    sampled_ratio = final_duration_dist[cls] / len(sampled_flow_data)
    preservation = min(sampled_ratio/original_ratio, original_ratio/sampled_ratio) * 100
    duration_preservation.append(preservation)

volume_preservation = []
for cls in final_volume_dist.index:
    original_ratio = volume_dist[cls] / len(flow_data)
    sampled_ratio = final_volume_dist[cls] / len(sampled_flow_data)
    preservation = min(sampled_ratio/original_ratio, original_ratio/sampled_ratio) * 100
    volume_preservation.append(preservation)

avg_duration_preservation = np.mean(duration_preservation)
avg_volume_preservation = np.mean(volume_preservation)
overall_preservation = (avg_duration_preservation + avg_volume_preservation) / 2

print(f"\n⚖️ 복합 층화 비율 보존도:")
print(f"   Duration: {avg_duration_preservation:.1f}%")
print(f"   Volume: {avg_volume_preservation:.1f}%") 
print(f"   전체 평균: {overall_preservation:.1f}% (100%에 가까울수록 완벽)")

print(f"\n✅ 복합 층화 샘플링 완료! Duration+Volume 동시 비율 보존으로 최고 품질 샘플 확보! 🎉")

🌍 전체 데이터 균등 층화 샘플링 + 4개 모델 하이퍼파라미터 튜닝

📂 1단계: 전체 데이터 파일 스캔
✓ 발견: task2_data/train_packet_data_50000.pkl
✓ 발견: task2_data/train_packet_data_100000.pkl
✓ 발견: task2_data/train_packet_data_150000.pkl
✓ 발견: task2_data/train_packet_data_200000.pkl
✓ 발견: task2_data/train_packet_data_250000.pkl
✓ 발견: task2_data/train_packet_data_300000.pkl
✓ 발견: task2_data/train_packet_data_350000.pkl
✓ 발견: task2_data/train_packet_data_400000.pkl
✓ 발견: task2_data/train_packet_data_450000.pkl
✓ 발견: task2_data/train_packet_data_500000.pkl
✓ 발견: task2_data/train_packet_data_550000.pkl
✓ 발견: task2_data/train_packet_data_600000.pkl

📊 총 12개 패킷 파일 발견

📈 Flow 데이터 로딩 및 클래스 분포 분석...
✓ Flow 데이터 크기: (600000, 11)

📊 전체 Duration Class 분포:
   Class 0: 23,407개 (3.9%)
   Class 1: 212,796개 (35.5%)
   Class 2: 140,943개 (23.5%)
   Class 3: 222,854개 (37.1%)

📊 전체 Volume Class 분포:
   Class 0: 205,670개 (34.3%)
   Class 1: 145,515개 (24.3%)
   Class 2: 213,478개 (35.6%)
   Class 3: 35,337개 (5.9%)

🎯 2단계: 균등 + 층화 샘플링 전략
🎯 목표 총 샘플 수: 50

In [10]:
# 5단계: 고급 특징 엔지니어링 (최대 3개 패킷 활용)
print(f"\n🔧 5단계: 고급 특징 엔지니어링")
print("="*50)

def extract_advanced_features(sampled_indices, packet_files):
    """
    샘플링된 인덱스를 기반으로 고급 특징 추출
    """
    print("🚀 고급 특징 추출 시작...")
    
    features_list = []
    valid_indices = []
    
    for global_idx in sampled_indices:
        try:
            # 어떤 파일에서 가져올지 결정
            file_idx = global_idx // 50000
            local_idx = global_idx % 50000
            
            if file_idx >= len(packet_files):
                continue
                
            # 해당 파일 로딩 (캐시 활용)
            file_path = packet_files[file_idx]
            
            if not hasattr(extract_advanced_features, 'cache'):
                extract_advanced_features.cache = {}
            
            if file_path not in extract_advanced_features.cache:
                print(f"   📂 로딩: {file_path}")
                extract_advanced_features.cache[file_path] = joblib.load(file_path)
            
            packet_data = extract_advanced_features.cache[file_path]
            
            # 로컬 인덱스가 범위를 벗어나면 스킵
            if local_idx >= len(packet_data):
                continue
                
            packet_df = packet_data[local_idx]
            valid_packets = packet_df.dropna()
            
            if len(valid_packets) >= 1:  # 최소 1개 패킷 필요
                features = extract_packet_features(valid_packets)
                features_list.append(features)
                valid_indices.append(global_idx)
                
        except Exception as e:
            continue
            
        if len(features_list) % 5000 == 0:
            print(f"      진행률: {len(features_list):,}개 완료")
    
    return features_list, valid_indices

def extract_packet_features(packets):
    """
    패킷들로부터 고급 특징 추출 (최대 3개 패킷만 사용)
    """
    features = {}
    num_packets = min(3, len(packets))
    packets = packets.iloc[:num_packets]
    
    # 숫자형 컬럼 확인
    numeric_cols = packets.select_dtypes(include=[np.number]).columns
    
    # === 기본 특징들 ===
    for col in numeric_cols:
        features[f'first_{col}'] = packets.iloc[0][col] if col in packets.columns else 0
        features[f'second_{col}'] = packets.iloc[1][col] if col in packets.columns and len(packets) > 1 else 0
    
    # === 🌟 통계 특징 (1~3개 패킷) ===
    if 'ip_len' in packets.columns:
        ip_lens = packets['ip_len'].values
        features['ip_len_mean_13'] = np.mean(ip_lens)
        features['ip_len_std_13'] = np.std(ip_lens) if len(ip_lens) > 1 else 0
        features['ip_len_max_13'] = np.max(ip_lens)
        features['ip_len_min_13'] = np.min(ip_lens)
        features['ip_len_range_13'] = np.max(ip_lens) - np.min(ip_lens)
        features['ip_len_median_13'] = np.median(ip_lens)
        
        # 변화 패턴
        if len(ip_lens) >= 3:
            diffs = np.diff(ip_lens)
            features['ip_len_trend'] = 1 if np.mean(diffs) > 0 else (-1 if np.mean(diffs) < 0 else 0)
            features['ip_len_volatility'] = np.std(diffs) if len(diffs) > 1 else 0
        else:
            features['ip_len_trend'] = 0
            features['ip_len_volatility'] = 0
    
    # 패킷 간 시간 통계
    if 'packet_capture_time' in packets.columns:
        try:
            times = pd.to_datetime(packets['packet_capture_time'])
            time_diffs = np.diff(times).astype('timedelta64[us]').astype(float)
            
            if len(time_diffs) > 0:
                features['inter_time_mean_13'] = np.mean(time_diffs)
                features['inter_time_std_13'] = np.std(time_diffs) if len(time_diffs) > 1 else 0
                features['inter_time_max_13'] = np.max(time_diffs)
                features['inter_time_min_13'] = np.min(time_diffs)
                features['timing_consistency'] = np.std(time_diffs) / (np.mean(time_diffs) + 1)
            else:
                for key in ['inter_time_mean_13', 'inter_time_std_13', 'inter_time_max_13', 
                           'inter_time_min_13', 'timing_consistency']:
                    features[key] = 0
        except:
            for key in ['inter_time_mean_13', 'inter_time_std_13', 'inter_time_max_13', 
                       'inter_time_min_13', 'timing_consistency']:
                features[key] = 0
    
    # TCP 효율성
    if 'tcp_len' in packets.columns and 'ip_len' in packets.columns:
        tcp_lens = packets['tcp_len'].values
        features['tcp_len_mean_13'] = np.mean(tcp_lens)
        features['tcp_len_std_13'] = np.std(tcp_lens) if len(tcp_lens) > 1 else 0
        features['tcp_len_sum_13'] = np.sum(tcp_lens)
        
        total_ip = np.sum(packets['ip_len'])
        total_tcp = np.sum(tcp_lens)
        features['tcp_efficiency_13'] = total_tcp / max(total_ip, 1)
    
    # === 🌟 TCP 플래그 패턴 특징 ===
    if 'tcp_flags' in packets.columns:
        flags = packets['tcp_flags'].values
        
        # 개별 플래그
        features['has_syn'] = int(any(flag & 0x02 for flag in flags))
        features['has_ack'] = int(any(flag & 0x10 for flag in flags))
        features['has_fin'] = int(any(flag & 0x01 for flag in flags))
        features['has_rst'] = int(any(flag & 0x04 for flag in flags))
        features['has_psh'] = int(any(flag & 0x08 for flag in flags))
        
        # 핸드셰이크 완전성
        if len(flags) >= 3:
            first_syn = (flags[0] & 0x02) != 0
            second_syn_ack = (flags[1] & 0x12) == 0x12
            third_ack = (flags[2] & 0x10) != 0
            features['is_handshake_complete'] = int(first_syn and second_syn_ack and third_ack)
            
            has_fin_ack = any((flag & 0x11) == 0x11 for flag in flags)
            features['is_graceful_close'] = int(has_fin_ack)
        else:
            features['is_handshake_complete'] = 0
            features['is_graceful_close'] = 0
        
        features['flag_diversity'] = len(set(flags))
        psh_count = sum(1 for flag in flags if flag & 0x08)
        features['push_frequency'] = psh_count / len(flags)
    
    # === 🌟 추가 고급 특징들 ===
    # 크기 변화 패턴
    if 'ip_len' in packets.columns and len(packets) >= 3:
        sizes = packets['ip_len'].values
        increases = sum(1 for i in range(1, len(sizes)) if sizes[i] > sizes[i-1])
        decreases = sum(1 for i in range(1, len(sizes)) if sizes[i] < sizes[i-1])
        
        features['size_increase_count'] = increases
        features['size_decrease_count'] = decreases
        features['size_stability'] = sum(1 for i in range(1, len(sizes)) if sizes[i] == sizes[i-1])
    
    # 비율 특징
    if 'tcp_len' in packets.columns and 'ip_len' in packets.columns:
        tcp_to_ip_ratio_1 = packets.iloc[0]['tcp_len'] / max(packets.iloc[0]['ip_len'], 1)
        features['tcp_to_ip_ratio_first'] = tcp_to_ip_ratio_1
        
        if len(packets) > 1:
            tcp_to_ip_ratio_2 = packets.iloc[1]['tcp_len'] / max(packets.iloc[1]['ip_len'], 1)
            features['tcp_to_ip_ratio_second'] = tcp_to_ip_ratio_2
    
    # 기존 호환성 특징
    features['inter_packet_time_us'] = features.get('inter_time_mean_13', 0)
    features['ip_len_diff'] = features.get('second_ip_len', 0) - features.get('first_ip_len', 0)
    
    return features

# 고급 특징 추출 실행
advanced_features_list, valid_global_indices = extract_advanced_features(sampled_indices, packet_files)

print(f"\n✅ 고급 특징 추출 완료!")
print(f"📊 추출된 특징 수: {len(advanced_features_list):,}개")
print(f"🎯 유효 인덱스 수: {len(valid_global_indices):,}개")

# 데이터프레임 생성
advanced_features_df = pd.DataFrame(advanced_features_list)
advanced_features_df = advanced_features_df.fillna(0)

# 대응하는 타겟 데이터
valid_flow_data = flow_data.iloc[valid_global_indices].copy()
valid_flow_data.reset_index(drop=True, inplace=True)

print(f"✓ 최종 특징 행렬 크기: {advanced_features_df.shape}")
print(f"✓ 타겟 데이터 크기: {valid_flow_data.shape}")

# 새로운 고급 특징들 확인
new_advanced_features = [col for col in advanced_features_df.columns 
                        if any(pattern in col for pattern in ['_13', 'handshake', 'efficiency', 
                                                             'consistency', 'diversity', 'frequency',
                                                             'stability', 'trend', 'volatility'])]

print(f"\n🌟 생성된 고급 특징들 ({len(new_advanced_features)}개):")
for i, feat in enumerate(new_advanced_features[:10], 1):  # 처음 10개만 표시
    print(f"   {i:2d}. 🧠 {feat}")
if len(new_advanced_features) > 10:
    print(f"   ... 외 {len(new_advanced_features)-10}개 더")

print(f"\n✅ 5단계 완료: 고급 특징 엔지니어링 완료! 🎉")


🔧 5단계: 고급 특징 엔지니어링
🚀 고급 특징 추출 시작...
   📂 로딩: task2_data/train_packet_data_50000.pkl
   📂 로딩: task2_data/train_packet_data_100000.pkl
   📂 로딩: task2_data/train_packet_data_100000.pkl
      진행률: 5,000개 완료
      진행률: 5,000개 완료
   📂 로딩: task2_data/train_packet_data_150000.pkl
   📂 로딩: task2_data/train_packet_data_150000.pkl
      진행률: 10,000개 완료
      진행률: 10,000개 완료
   📂 로딩: task2_data/train_packet_data_250000.pkl
   📂 로딩: task2_data/train_packet_data_250000.pkl
      진행률: 15,000개 완료
      진행률: 15,000개 완료
   📂 로딩: task2_data/train_packet_data_300000.pkl
   📂 로딩: task2_data/train_packet_data_300000.pkl
      진행률: 20,000개 완료
      진행률: 20,000개 완료
   📂 로딩: task2_data/train_packet_data_350000.pkl
   📂 로딩: task2_data/train_packet_data_350000.pkl
   📂 로딩: task2_data/train_packet_data_400000.pkl
   📂 로딩: task2_data/train_packet_data_400000.pkl
      진행률: 25,000개 완료
      진행률: 25,000개 완료
   📂 로딩: task2_data/train_packet_data_450000.pkl
   📂 로딩: task2_data/train_packet_data_450000.pkl
      진행률: 

In [ ]:
# 6단계: 5개 모델 100회 하이퍼파라미터 튜닝 (GPU 가속 + 클래스 가중치 + Optuna)
print(f"\n🚀 6단계: 5개 모델 100회 하이퍼파라미터 튜닝")
print("="*80)

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
import subprocess

# Optuna 설치 및 임포트
try:
    import optuna
    optuna.logging.set_verbosity(optuna.logging.INFO)  # 진행상황 표시
    OPTUNA_AVAILABLE = True
    print("✅ Optuna를 사용한 지능형 하이퍼파라미터 탐색")
except ImportError:
    OPTUNA_AVAILABLE = False
    print("❌ Optuna 없음 - 기존 랜덤 서치 사용")
    print("   설치: pip install optuna")

# GPU 확인 함수 (강화 버전)
def check_gpu_advanced():
    gpu_status = {}
    
    # NVIDIA GPU 확인
    try:
        result = subprocess.run(['nvidia-smi'], capture_output=True, text=True, timeout=5)
        gpu_status['nvidia'] = result.returncode == 0
    except:
        gpu_status['nvidia'] = False
    
    # CatBoost GPU 테스트
    try:
        test_cat = CatBoostClassifier(task_type='GPU', iterations=1, verbose=False)
        test_cat.fit([[1, 2], [3, 4]], [0, 1])
        gpu_status['catboost'] = True
    except:
        gpu_status['catboost'] = False
    
    # XGBoost GPU 테스트
    try:
        test_xgb = xgb.XGBClassifier(tree_method='gpu_hist', n_estimators=1)
        test_xgb.fit([[1, 2], [3, 4]], [0, 1])
        gpu_status['xgboost'] = True
    except:
        gpu_status['xgboost'] = False
    
    # LightGBM GPU 테스트
    try:
        test_lgb = lgb.LGBMClassifier(device='gpu', n_estimators=1, verbose=-1)
        test_lgb.fit([[1, 2], [3, 4]], [0, 1])
        gpu_status['lightgbm'] = True
    except:
        gpu_status['lightgbm'] = False
    
    return gpu_status

gpu_status = check_gpu_advanced()
print(f"🖥️ GPU 상태:")
print(f"   NVIDIA GPU: {'✅' if gpu_status['nvidia'] else '❌'}")
print(f"   CatBoost GPU: {'✅' if gpu_status['catboost'] else '❌'}")
print(f"   XGBoost GPU: {'✅' if gpu_status['xgboost'] else '❌'}")
print(f"   LightGBM GPU: {'✅' if gpu_status['lightgbm'] else '❌'}")

# 데이터 준비
X = advanced_features_df
y_duration = valid_flow_data['duration_class']
y_volume = valid_flow_data['volume_class']

# 데이터 분할
X_train, X_test, y_duration_train, y_duration_test = train_test_split(
    X, y_duration, test_size=0.2, random_state=42, stratify=y_duration
)
_, _, y_volume_train, y_volume_test = train_test_split(
    X, y_volume, test_size=0.2, random_state=42, stratify=y_volume
)

print(f"\n📊 데이터 준비 완료:")
print(f"   학습 데이터: {X_train.shape[0]:,}개")
print(f"   검증 데이터: {X_test.shape[0]:,}개") 
print(f"   특징 수: {X_train.shape[1]:,}개")

# 클래스 가중치 계산
duration_classes = np.unique(y_duration_train)
duration_weights = compute_class_weight('balanced', classes=duration_classes, y=y_duration_train)
duration_class_weights = {int(cls): weight for cls, weight in zip(duration_classes, duration_weights)}

volume_classes = np.unique(y_volume_train)
volume_weights = compute_class_weight('balanced', classes=volume_classes, y=y_volume_train)
volume_class_weights = {int(cls): weight for cls, weight in zip(volume_classes, volume_weights)}

print(f"\n⚖️ 클래스 가중치:")
print(f"   Duration: {duration_class_weights}")
print(f"   Volume: {volume_class_weights}")

# 결과 저장
results = {
    'model': [], 'task': [], 'accuracy': [], 'precision': [], 
    'recall': [], 'f1_weighted': [], 'f1_macro': [], 'best_params': []
}

def evaluate_model(model, X_test, y_test, model_name, task_name, best_params):
    """모델 평가"""
    y_pred = model.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1_weighted = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    f1_macro = f1_score(y_test, y_pred, average='macro', zero_division=0)
    
    results['model'].append(model_name)
    results['task'].append(task_name)
    results['accuracy'].append(accuracy)
    results['precision'].append(precision)
    results['recall'].append(recall)
    results['f1_weighted'].append(f1_weighted)
    results['f1_macro'].append(f1_macro)
    results['best_params'].append(str(best_params)[:100])  # 길이 제한
    
    print(f"   정확도: {accuracy:.4f} | 정밀도: {precision:.4f} | 재현율: {recall:.4f}")
    print(f"   F1(weighted): {f1_weighted:.4f} | F1(macro): {f1_macro:.4f}")

# 클래스 가중치를 샘플 가중치로 변환하는 함수
def create_sample_weights(y, class_weights):
    """클래스 가중치를 샘플별 가중치 배열로 변환"""
    return np.array([class_weights[cls] for cls in y])

# Optuna 기반 지능형 하이퍼파라미터 튜닝 클래스
class OptunaHyperparameterSearch:
    def __init__(self, model_class, param_ranges, n_trials, cv, scoring, random_state, gpu_available=False):
        self.model_class = model_class
        self.param_ranges = param_ranges
        self.n_trials = n_trials
        self.cv = cv
        self.scoring = scoring
        self.random_state = random_state
        self.gpu_available = gpu_available
        self.best_estimator_ = None
        self.best_params_ = None
        self.best_score_ = None
        
    def objective(self, trial):
        # 파라미터 샘플링 (Optuna 권장 방식: 이산값은 categorical로 처리)
        params = {}
        for param_name, param_config in self.param_ranges.items():
            if isinstance(param_config, list):
                # 🔬 프로 팁: 모든 리스트는 categorical로 처리하여 정확한 탐색 공간 제어
                params[param_name] = trial.suggest_categorical(param_name, param_config)
        
        # 모델별 특수 설정
        model_params = {}
        if self.model_class.__name__ == 'CatBoostClassifier':
            model_params.update({
                'task_type': 'GPU' if self.gpu_available else 'CPU',
                'random_seed': self.random_state,
                'verbose': False,
                'eval_metric': 'MultiClass'
            })
        elif self.model_class.__name__ == 'XGBClassifier':
            model_params.update({
                'tree_method': 'gpu_hist' if self.gpu_available else 'hist',
                'random_state': self.random_state,
                'n_jobs': -1 if not self.gpu_available else 1,
                'eval_metric': 'mlogloss'
            })
        elif self.model_class.__name__ == 'LGBMClassifier':
            model_params.update({
                'device': 'gpu' if self.gpu_available else 'cpu',
                'random_state': self.random_state,
                'verbose': -1,
                'n_jobs': -1 if not self.gpu_available else 1
            })
        elif self.model_class.__name__ in ['RandomForestClassifier', 'ExtraTreesClassifier']:
            model_params.update({
                'random_state': self.random_state,
                'n_jobs': -1
            })
        
        # 모델 생성
        model = self.model_class(**{**model_params, **params})
        
        # 교차 검증 - XGBoost sample_weight 문제 해결
        from sklearn.model_selection import StratifiedKFold
        from sklearn.metrics import f1_score
        import numpy as np
        
        # 수동 교차 검증으로 sample_weight 문제 해결
        skf = StratifiedKFold(n_splits=self.cv, shuffle=True, random_state=self.random_state)
        scores = []
        
        for train_idx, val_idx in skf.split(self.X, self.y):
            X_train_fold, X_val_fold = self.X.iloc[train_idx], self.X.iloc[val_idx]
            y_train_fold, y_val_fold = self.y.iloc[train_idx], self.y.iloc[val_idx]
            
            # 모델 학습
            if hasattr(self, 'fit_params') and self.fit_params:
                # XGBoost의 경우 sample_weight 적용
                if 'sample_weight' in self.fit_params:
                    fold_sample_weights = self.fit_params['sample_weight'][train_idx]
                    model.fit(X_train_fold, y_train_fold, sample_weight=fold_sample_weights)
                else:
                    model.fit(X_train_fold, y_train_fold)
            else:
                model.fit(X_train_fold, y_train_fold)
            
            # 예측 및 점수 계산
            y_pred_fold = model.predict(X_val_fold)
            score = f1_score(y_val_fold, y_pred_fold, average='weighted')
            scores.append(score)
        
        return np.mean(scores)
    
    def fit(self, X, y, class_weights=None):
        self.X = X
        self.y = y
        
        # 클래스 가중치 처리
        self.fit_params = {}
        if class_weights is not None:
            if self.model_class.__name__ == 'XGBClassifier':
                # XGBoost는 sample_weight 사용
                sample_weights = create_sample_weights(y, class_weights)
                self.fit_params = {'sample_weight': sample_weights}
            elif self.model_class.__name__ == 'CatBoostClassifier':
                # CatBoost는 class_weights 파라미터 사용
                self.class_weights = class_weights
            else:
                # 다른 모델들은 class_weight='balanced' 사용
                pass
        
        if OPTUNA_AVAILABLE:
            # Optuna 사용 - 진행상황 콜백 추가
            def progress_callback(study, trial):
                print(f"   진행률: ({trial.number + 1:3d}/{self.n_trials}) F1: {trial.value:.4f} | 최고: {study.best_value:.4f}")
            
            study = optuna.create_study(
                direction='maximize',
                sampler=optuna.samplers.TPESampler(seed=self.random_state)
            )
            
            study.optimize(
                self.objective, 
                n_trials=self.n_trials, 
                callbacks=[progress_callback],
                show_progress_bar=False
            )
            
            self.best_params_ = study.best_params
            self.best_score_ = study.best_value
        else:
            # 기존 랜덤 서치 사용 - 진행상황 표시 추가
            from sklearn.model_selection import ParameterSampler
            param_list = list(ParameterSampler(self.param_ranges, n_iter=self.n_trials, random_state=self.random_state))
            
            best_score = -np.inf
            best_params = None
            
            for i, params in enumerate(param_list, 1):
                print(f"   진행률: ({i:3d}/{self.n_trials}) 테스트 중...", end=' ')
                
                try:
                    model_params = {}
                    if self.model_class.__name__ == 'CatBoostClassifier':
                        model_params.update({
                            'task_type': 'GPU' if self.gpu_available else 'CPU',
                            'random_seed': self.random_state,
                            'verbose': False,
                            'eval_metric': 'MultiClass'
                        })
                    elif self.model_class.__name__ == 'XGBClassifier':
                        model_params.update({
                            'tree_method': 'gpu_hist' if self.gpu_available else 'hist',
                            'random_state': self.random_state,
                            'n_jobs': -1 if not self.gpu_available else 1,
                            'eval_metric': 'mlogloss'
                        })
                    elif self.model_class.__name__ == 'LGBMClassifier':
                        model_params.update({
                            'device': 'gpu' if self.gpu_available else 'cpu',
                            'random_state': self.random_state,
                            'verbose': -1,
                            'n_jobs': -1 if not self.gpu_available else 1
                        })
                    elif self.model_class.__name__ in ['RandomForestClassifier', 'ExtraTreesClassifier']:
                        model_params.update({
                            'random_state': self.random_state,
                            'n_jobs': -1
                        })
                    
                    model = self.model_class(**{**model_params, **params})
                    
                    # 수동 교차 검증으로 sample_weight 문제 해결
                    from sklearn.model_selection import StratifiedKFold
                    from sklearn.metrics import f1_score
                    import numpy as np
                    
                    skf = StratifiedKFold(n_splits=self.cv, shuffle=True, random_state=self.random_state)
                    scores = []
                    
                    for train_idx, val_idx in skf.split(X, y):
                        X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
                        y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]
                        
                        # 각 모델별 복사본 생성
                        fold_model = self.model_class(**{**model_params, **params})
                        
                        # 모델 학습
                        if hasattr(self, 'fit_params') and self.fit_params:
                            # XGBoost의 경우 sample_weight 적용
                            if 'sample_weight' in self.fit_params:
                                fold_sample_weights = self.fit_params['sample_weight'][train_idx]
                                fold_model.fit(X_train_fold, y_train_fold, sample_weight=fold_sample_weights)
                            else:
                                fold_model.fit(X_train_fold, y_train_fold)
                        else:
                            fold_model.fit(X_train_fold, y_train_fold)
                        
                        # 예측 및 점수 계산
                        y_pred_fold = fold_model.predict(X_val_fold)
                        score = f1_score(y_val_fold, y_pred_fold, average='weighted')
                        scores.append(score)
                    
                    avg_score = np.mean(scores)
                    
                    print(f"F1: {avg_score:.4f} | 최고: {best_score:.4f}")
                    
                    if avg_score > best_score:
                        best_score = avg_score
                        best_params = params
                except Exception as e:
                    print(f"실패")
                    continue
            
            self.best_params_ = best_params
            self.best_score_ = best_score
        
        # 최적 모델 생성 및 학습
        model_params = {}
        if self.model_class.__name__ == 'CatBoostClassifier':
            model_params.update({
                'task_type': 'GPU' if self.gpu_available else 'CPU',
                'class_weights': class_weights if class_weights else None,
                'random_seed': self.random_state,
                'verbose': False,
                'eval_metric': 'MultiClass'
            })
        elif self.model_class.__name__ == 'XGBClassifier':
            model_params.update({
                'tree_method': 'gpu_hist' if self.gpu_available else 'hist',
                'random_state': self.random_state,
                'n_jobs': -1 if not self.gpu_available else 1,
                'eval_metric': 'mlogloss'
            })
        elif self.model_class.__name__ == 'LGBMClassifier':
            model_params.update({
                'device': 'gpu' if self.gpu_available else 'cpu',
                'class_weight': 'balanced',
                'random_state': self.random_state,
                'verbose': -1,
                'n_jobs': -1 if not self.gpu_available else 1
            })
        elif self.model_class.__name__ in ['RandomForestClassifier', 'ExtraTreesClassifier']:
            model_params.update({
                'class_weight': 'balanced',
                'random_state': self.random_state,
                'n_jobs': -1
            })
        
        self.best_estimator_ = self.model_class(**{**model_params, **self.best_params_})
        
        # XGBoost의 경우 sample_weight와 함께 학습
        if self.model_class.__name__ == 'XGBClassifier' and class_weights:
            sample_weights = create_sample_weights(y, class_weights)
            self.best_estimator_.fit(X, y, sample_weight=sample_weights)
        else:
            self.best_estimator_.fit(X, y)
        
        return self

print("\n" + "="*80)
print("🐱 1. CatBoost 하이퍼파라미터 튜닝 (100회)")
print("="*80)

# 🔬 프로 팁: 이산값들은 정확히 지정된 값들만 탐색하도록 설계
catboost_param_ranges = {
    'iterations': [200, 300, 400, 500, 600],           # 정확히 이 값들만 탐색
    'depth': [6, 8, 10, 12, 14],                       # 7, 9, 11, 13은 제외
    'learning_rate': [0.01, 0.03, 0.05, 0.08, 0.1, 0.15],  # 실험에서 검증된 값들만
    'l2_leaf_reg': [1, 3, 5, 7, 10, 15],
    'border_count': [64, 128, 254],                    # CatBoost 권장값들만
    'bagging_temperature': [0.5, 1.0, 1.5, 2.0],
    'bootstrap_type': ['Bayesian', 'Bernoulli', 'MVS'],
    'leaf_estimation_method': ['Newton', 'Gradient']
}

# Duration 분류
print("\n🎯 CatBoost - Duration 분류 튜닝...")
start_time = time.time()

catboost_search_duration = OptunaHyperparameterSearch(
    model_class=CatBoostClassifier,
    param_ranges=catboost_param_ranges,
    n_trials=100,
    cv=3,
    scoring='f1_weighted',
    random_state=42,
    gpu_available=gpu_status['catboost']
)

catboost_search_duration.fit(X_train, y_duration_train, class_weights=duration_class_weights)
print(f"⏱️ 소요시간: {time.time() - start_time:.1f}초")
print(f"🏆 최적 파라미터: {catboost_search_duration.best_params_}")
print(f"🎯 교차검증 점수: {catboost_search_duration.best_score_:.4f}")
evaluate_model(catboost_search_duration.best_estimator_, X_test, y_duration_test, 
               'CatBoost', 'Duration', catboost_search_duration.best_params_)

# Volume 분류
print("\n📦 CatBoost - Volume 분류 튜닝...")
start_time = time.time()

catboost_search_volume = OptunaHyperparameterSearch(
    model_class=CatBoostClassifier,
    param_ranges=catboost_param_ranges,
    n_trials=100,
    cv=3,
    scoring='f1_weighted',
    random_state=42,
    gpu_available=gpu_status['catboost']
)

catboost_search_volume.fit(X_train, y_volume_train, class_weights=volume_class_weights)
print(f"⏱️ 소요시간: {time.time() - start_time:.1f}초")
print(f"🏆 최적 파라미터: {catboost_search_volume.best_params_}")
print(f"🎯 교차검증 점수: {catboost_search_volume.best_score_:.4f}")
evaluate_model(catboost_search_volume.best_estimator_, X_test, y_volume_test, 
               'CatBoost', 'Volume', catboost_search_volume.best_params_)

print("\n" + "="*80)
print("🌲 2. RandomForest 하이퍼파라미터 튜닝 (100회)")
print("="*80)

# 🔬 각 파라미터마다 의미 있는 특정 값들만 선별하여 정밀한 탐색
rf_param_ranges = {
    'n_estimators': [100, 200, 300, 500, 700, 1000],     # 계산 비용 고려한 선별값
    'max_depth': [10, 15, 20, 25, 30, None],             # 과적합 방지 최적값들
    'min_samples_split': [2, 5, 10, 15, 20],             # 일반적인 분할 기준값들
    'min_samples_leaf': [1, 2, 4, 8, 12],                # 리프 노드 최소 샘플 기준
    'max_features': ['sqrt', 'log2', 0.2, 0.3, 0.5, 0.7], # 특징 선택 전략
    'bootstrap': [True, False],
    'max_samples': [0.7, 0.8, 0.9, 1.0]                  # 부트스트랩 샘플 비율
}

# Duration 분류  
print("\n🎯 RandomForest - Duration 분류 튜닝...")
start_time = time.time()

rf_search_duration = OptunaHyperparameterSearch(
    model_class=RandomForestClassifier,
    param_ranges=rf_param_ranges,
    n_trials=100,
    cv=3,
    scoring='f1_weighted',
    random_state=42,
    gpu_available=False  # RandomForest는 GPU 미지원
)

rf_search_duration.fit(X_train, y_duration_train, class_weights=None)  # class_weight='balanced' 내장
print(f"⏱️ 소요시간: {time.time() - start_time:.1f}초")
print(f"🏆 최적 파라미터: {rf_search_duration.best_params_}")
print(f"🎯 교차검증 점수: {rf_search_duration.best_score_:.4f}")
evaluate_model(rf_search_duration.best_estimator_, X_test, y_duration_test, 
               'RandomForest', 'Duration', rf_search_duration.best_params_)

# Volume 분류
print("\n📦 RandomForest - Volume 분류 튜닝...")
start_time = time.time()

rf_search_volume = OptunaHyperparameterSearch(
    model_class=RandomForestClassifier,
    param_ranges=rf_param_ranges,
    n_trials=100,
    cv=3,
    scoring='f1_weighted',
    random_state=42,
    gpu_available=False
)

rf_search_volume.fit(X_train, y_volume_train, class_weights=None)
print(f"⏱️ 소요시간: {time.time() - start_time:.1f}초")
print(f"🏆 최적 파라미터: {rf_search_volume.best_params_}")
print(f"🎯 교차검증 점수: {rf_search_volume.best_score_:.4f}")
evaluate_model(rf_search_volume.best_estimator_, X_test, y_volume_test, 
               'RandomForest', 'Volume', rf_search_volume.best_params_)


🚀 6단계: 5개 모델 100회 하이퍼파라미터 튜닝
✅ Optuna를 사용한 지능형 하이퍼파라미터 탐색


[I 2025-08-07 23:52:59,064] A new study created in memory with name: no-name-793dd8bb-e312-4f41-afcd-3ec8e2894613
[W 2025-08-07 23:52:59,067] Trial 0 failed with parameters: {'iterations': 300, 'depth': 10, 'learning_rate': 0.03, 'l2_leaf_reg': 10, 'border_count': 254, 'bagging_temperature': 0.5, 'bootstrap_type': 'Bernoulli', 'leaf_estimation_method': 'Gradient'} because of the following error: TypeError("got an unexpected keyword argument 'fit_params'").
Traceback (most recent call last):
  File "c:\Users\hg226\AppData\Local\Programs\Python\Python313\Lib\site-packages\optuna\study\_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\hg226\AppData\Local\Temp\ipykernel_5696\2007194405.py", line 185, in objective
    scores = cross_val_score(
        model, self.X, self.y, cv=self.cv, scoring=self.scoring,
        fit_params=self.fit_params if hasattr(self, 'fit_params') else None,
        n_jobs=1
    )
  File "c:\Users\hg226\AppData\Local\Programs\

🖥️ GPU 상태:
   NVIDIA GPU: ✅
   CatBoost GPU: ✅
   XGBoost GPU: ✅
   LightGBM GPU: ✅

📊 데이터 준비 완료:
   학습 데이터: 33,328개
   검증 데이터: 8,332개
   특징 수: 44개

⚖️ 클래스 가중치:
   Duration: {0: np.float64(6.345773038842346), 1: np.float64(0.701405842242613), 2: np.float64(1.0609957977842863), 3: np.float64(0.678335911422291)}
   Volume: {0: np.float64(0.723389477339816), 1: np.float64(1.0242163491087892), 2: np.float64(0.7099522835719154), 3: np.float64(4.29706034038164)}

🐱 1. CatBoost 하이퍼파라미터 튜닝 (100회)

🎯 CatBoost - Duration 분류 튜닝...


TypeError: got an unexpected keyword argument 'fit_params'

In [ ]:
print("\n" + "="*80)
print("💡 3. LightGBM 하이퍼파라미터 튜닝 (100회)")
print("="*80)

# 🔬 LightGBM 특성을 고려한 최적화된 탐색 공간
lgb_param_ranges = {
    'n_estimators': [100, 200, 300, 500, 700, 1000],     # 조기 종료와 함께 사용
    'max_depth': [6, 8, 10, 12, 15, 20, -1],             # -1은 제한 없음
    'learning_rate': [0.01, 0.03, 0.05, 0.08, 0.1, 0.15, 0.2], # 학습률 세밀 조정
    'num_leaves': [31, 50, 100, 200, 300, 500],          # 트리 복잡도 제어
    'min_child_samples': [10, 20, 30, 50, 100],          # 과적합 방지
    'subsample': [0.7, 0.8, 0.9, 1.0],                   # 샘플링 비율
    'colsample_bytree': [0.7, 0.8, 0.9, 1.0],           # 특징 샘플링
    'reg_alpha': [0, 0.1, 0.5, 1.0, 2.0],               # L1 정규화
    'reg_lambda': [0, 0.1, 0.5, 1.0, 2.0]               # L2 정규화
}

# Duration 분류
print("\n🎯 LightGBM - Duration 분류 튜닝...")
start_time = time.time()

lgb_search_duration = OptunaHyperparameterSearch(
    model_class=lgb.LGBMClassifier,
    param_ranges=lgb_param_ranges,
    n_trials=100,
    cv=3,
    scoring='f1_weighted',
    random_state=42,
    gpu_available=gpu_status['lightgbm']
)

lgb_search_duration.fit(X_train, y_duration_train, class_weights=None)  # class_weight='balanced' 내장
print(f"⏱️ 소요시간: {time.time() - start_time:.1f}초")
print(f"🏆 최적 파라미터: {lgb_search_duration.best_params_}")
print(f"🎯 교차검증 점수: {lgb_search_duration.best_score_:.4f}")
evaluate_model(lgb_search_duration.best_estimator_, X_test, y_duration_test, 
               'LightGBM', 'Duration', lgb_search_duration.best_params_)

# Volume 분류
print("\n📦 LightGBM - Volume 분류 튜닝...")
start_time = time.time()

lgb_search_volume = OptunaHyperparameterSearch(
    model_class=lgb.LGBMClassifier,
    param_ranges=lgb_param_ranges,
    n_trials=100,
    cv=3,
    scoring='f1_weighted',
    random_state=42,
    gpu_available=gpu_status['lightgbm']
)

lgb_search_volume.fit(X_train, y_volume_train, class_weights=None)
print(f"⏱️ 소요시간: {time.time() - start_time:.1f}초")
print(f"🏆 최적 파라미터: {lgb_search_volume.best_params_}")
print(f"🎯 교차검증 점수: {lgb_search_volume.best_score_:.4f}")
evaluate_model(lgb_search_volume.best_estimator_, X_test, y_volume_test, 
               'LightGBM', 'Volume', lgb_search_volume.best_params_)

print("\n" + "="*80)
print("🚀 4. XGBoost 하이퍼파라미터 튜닝 (100회) - 🎯 클래스 가중치 완벽 지원!")
print("="*80)

# 🔬 XGBoost 전용 최적화: 각 파라미터의 상호작용을 고려한 값들
xgb_param_ranges = {
    'n_estimators': [100, 200, 300, 500, 700, 1000],     # early_stopping과 조화
    'max_depth': [3, 6, 8, 10, 12, 15],                  # XGBoost 권장 깊이 범위
    'learning_rate': [0.01, 0.03, 0.05, 0.08, 0.1, 0.15, 0.2], # eta 최적값들
    'subsample': [0.7, 0.8, 0.9, 1.0],                   # 행 샘플링
    'colsample_bytree': [0.7, 0.8, 0.9, 1.0],           # 열 샘플링
    'gamma': [0, 0.1, 0.5, 1.0, 2.0],                   # 최소 분할 loss
    'reg_alpha': [0, 0.1, 0.5, 1.0, 2.0],               # L1 정규화
    'reg_lambda': [0, 0.1, 0.5, 1.0, 2.0],              # L2 정규화
    'min_child_weight': [1, 3, 5, 7, 10]                # 자식 노드 가중치 합
}

# Duration 분류
print("\n🎯 XGBoost - Duration 분류 튜닝...")
start_time = time.time()

xgb_search_duration = OptunaHyperparameterSearch(
    model_class=xgb.XGBClassifier,
    param_ranges=xgb_param_ranges,
    n_trials=100,
    cv=3,
    scoring='f1_weighted',
    random_state=42,
    gpu_available=gpu_status['xgboost']
)

xgb_search_duration.fit(X_train, y_duration_train, class_weights=duration_class_weights)
print(f"⏱️ 소요시간: {time.time() - start_time:.1f}초")
print(f"🏆 최적 파라미터: {xgb_search_duration.best_params_}")
print(f"🎯 교차검증 점수: {xgb_search_duration.best_score_:.4f}")
evaluate_model(xgb_search_duration.best_estimator_, X_test, y_duration_test, 
               'XGBoost', 'Duration', xgb_search_duration.best_params_)

# Volume 분류
print("\n📦 XGBoost - Volume 분류 튜닝...")
start_time = time.time()

xgb_search_volume = OptunaHyperparameterSearch(
    model_class=xgb.XGBClassifier,
    param_ranges=xgb_param_ranges,
    n_trials=100,
    cv=3,
    scoring='f1_weighted',
    random_state=42,
    gpu_available=gpu_status['xgboost']
)

xgb_search_volume.fit(X_train, y_volume_train, class_weights=volume_class_weights)
print(f"⏱️ 소요시간: {time.time() - start_time:.1f}초")
print(f"🏆 최적 파라미터: {xgb_search_volume.best_params_}")
print(f"🎯 교차검증 점수: {xgb_search_volume.best_score_:.4f}")
evaluate_model(xgb_search_volume.best_estimator_, X_test, y_volume_test, 
               'XGBoost', 'Volume', xgb_search_volume.best_params_)

print("\n" + "="*80)
print("🌳 5. ExtraTrees 하이퍼파라미터 튜닝 (100회)")
print("="*80)

# 🔬 ExtraTrees 특성: 무작위성이 높으므로 안정적인 값들 위주로 탐색
et_param_ranges = {
    'n_estimators': [100, 200, 300, 500, 700, 1000],     # 높은 무작위성 보상
    'max_depth': [10, 15, 20, 25, 30, None],             # 깊이 제한 전략
    'min_samples_split': [2, 5, 10, 15, 20],             # 분할 임계값
    'min_samples_leaf': [1, 2, 4, 8, 12],                # 리프 크기 제어
    'max_features': ['sqrt', 'log2', 0.2, 0.3, 0.5, 0.7], # 특징 선택 다양성
    'bootstrap': [True, False],                           # 샘플링 방식
    'max_samples': [0.7, 0.8, 0.9, 1.0]                  # 부트스트랩 크기
}

# Duration 분류
print("\n🎯 ExtraTrees - Duration 분류 튜닝...")
start_time = time.time()

et_search_duration = OptunaHyperparameterSearch(
    model_class=ExtraTreesClassifier,
    param_ranges=et_param_ranges,
    n_trials=100,
    cv=3,
    scoring='f1_weighted',
    random_state=42,
    gpu_available=False  # ExtraTrees는 GPU 미지원
)

et_search_duration.fit(X_train, y_duration_train, class_weights=None)  # class_weight='balanced' 내장
print(f"⏱️ 소요시간: {time.time() - start_time:.1f}초")
print(f"🏆 최적 파라미터: {et_search_duration.best_params_}")
print(f"🎯 교차검증 점수: {et_search_duration.best_score_:.4f}")
evaluate_model(et_search_duration.best_estimator_, X_test, y_duration_test, 
               'ExtraTrees', 'Duration', et_search_duration.best_params_)

# Volume 분류
print("\n📦 ExtraTrees - Volume 분류 튜닝...")
start_time = time.time()

et_search_volume = OptunaHyperparameterSearch(
    model_class=ExtraTreesClassifier,
    param_ranges=et_param_ranges,
    n_trials=100,
    cv=3,
    scoring='f1_weighted',
    random_state=42,
    gpu_available=False
)

et_search_volume.fit(X_train, y_volume_train, class_weights=None)
print(f"⏱️ 소요시간: {time.time() - start_time:.1f}초")
print(f"🏆 최적 파라미터: {et_search_volume.best_params_}")
print(f"🎯 교차검증 점수: {et_search_volume.best_score_:.4f}")
evaluate_model(et_search_volume.best_estimator_, X_test, y_volume_test, 
               'ExtraTrees', 'Volume', et_search_volume.best_params_)

print("\n" + "="*80)
print("🏆 최종 성능 비교 결과")
print("="*80)

# 결과 DataFrame 생성
final_results_df = pd.DataFrame(results)

# Duration 결과
print("\n🎯 Duration 분류 결과:")
duration_results = final_results_df[final_results_df['task'] == 'Duration'].copy()
duration_results = duration_results.sort_values('f1_weighted', ascending=False)
print(f"{'순위':<4} {'모델':<12} {'정확도':<8} {'정밀도':<8} {'재현율':<8} {'F1(W)':<8} {'F1(M)':<8}")
print("-" * 60)
for i, (_, row) in enumerate(duration_results.iterrows(), 1):
    print(f"{i:<4} {row['model']:<12} {row['accuracy']:<8.4f} {row['precision']:<8.4f} "
          f"{row['recall']:<8.4f} {row['f1_weighted']:<8.4f} {row['f1_macro']:<8.4f}")

print("\n📦 Volume 분류 결과:")
volume_results = final_results_df[final_results_df['task'] == 'Volume'].copy()
volume_results = volume_results.sort_values('f1_weighted', ascending=False)
print(f"{'순위':<4} {'모델':<12} {'정확도':<8} {'정밀도':<8} {'재현율':<8} {'F1(W)':<8} {'F1(M)':<8}")
print("-" * 60)
for i, (_, row) in enumerate(volume_results.iterrows(), 1):
    print(f"{i:<4} {row['model']:<12} {row['accuracy']:<8.4f} {row['precision']:<8.4f} "
          f"{row['recall']:<8.4f} {row['f1_weighted']:<8.4f} {row['f1_macro']:<8.4f}")

print("\n📈 전체 최고 성능:")
best_duration = duration_results.iloc[0]
best_volume = volume_results.iloc[0]
print(f"Duration 최고: {best_duration['model']} (F1-weighted: {best_duration['f1_weighted']:.4f})")
print(f"Volume 최고: {best_volume['model']} (F1-weighted: {best_volume['f1_weighted']:.4f})")

print(f"\n✅ 전체 하이퍼파라미터 튜닝 완료!")
print(f"   - 5개 모델 × 2개 태스크 × 100회 = 총 1,000회 튜닝")
print(f"   - GPU 가속 활용: {sum(gpu_status.values())}개 모델")
print(f"   - 고급 특징 엔지니어링 + 클래스 가중치 적용")
print(f"   - {'🧠 Optuna 지능형 탐색' if OPTUNA_AVAILABLE else '🎲 랜덤 탐색'} 사용")
print(f"   - 🎯 XGBoost 클래스 가중치 완벽 지원!")
print(f"   - ⚖️ Duration+Volume 복합 층화 샘플링")
print(f"   - 🔬 정밀한 탐색공간: suggest_categorical로 최적화")